<a href="https://colab.research.google.com/github/deepak-karkala/chandler-bing-comment-generator/blob/main/data_preprocessing_sitcom_characters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load script dataset

In [1]:
"""
!pip install kaggle
!kaggle datasets download -d ryanstonebraker/friends-transcript
import zipfile
with zipfile.ZipFile('friends-transcript.zip', 'r') as zip_ref:
    zip_ref.extractall()
"""

"\n!pip install kaggle\n!kaggle datasets download -d ryanstonebraker/friends-transcript\nimport zipfile\nwith zipfile.ZipFile('friends-transcript.zip', 'r') as zip_ref:\n    zip_ref.extractall()\n"

In [1]:
from tqdm import tqdm
import pandas as pd
import re
import json
from pydantic import BaseModel

In [7]:
input_data_path = "../datasets/friends/friends_quotes.csv"
batch_ip_path_part1 = "../datasets/friends/batch_friends_part1.jsonl"
batch_ip_path_part2 = "../datasets/friends/batch_friends_part2.jsonl"

batch_op_path_part1 = "../datasets/friends/batch_output_friends_part1.jsonl"

character_name = "Chandler"
content_system = """You are given a few dialogues from the famous TV sitcom FRIENDS. You have two tasks. First, you have to summarize the dialogues into context without any names of characters. Second, you have to then write a dialogue as an appropriate response to the given context, the dialogue should be in the style of Chandler Bing, one of the characters from FRIENDS. Keep in mind that Chandler Bing’s humor is marked by a unique blend of sarcasm, self-deprecation, and quick wit. He tends to make jokes that deflect serious or emotional moments, often using his dry, sarcastic tone.  His style is heavily reliant on irony, often delivering punchlines that are deliberately over-the-top or nonsensical. Extract these two things and return them as structured output specified as Pydantic schema."""
num_prev_dialogues = 12 # For generating context

class ContextDialogue(BaseModel):
    context: str
    dialog: str

In [8]:
df = pd.read_csv(input_data_path)
print(df.head())

     author  episode_number           episode_title  \
0    Monica             1.0  Monica Gets A Roommate   
1      Joey             1.0  Monica Gets A Roommate   
2  Chandler             1.0  Monica Gets A Roommate   
3    Phoebe             1.0  Monica Gets A Roommate   
4    Phoebe             1.0  Monica Gets A Roommate   

                                               quote  quote_order  season  
0  There's nothing to tell! He's just some guy I ...          0.0     1.0  
1  C'mon, you're going out with the guy! There's ...          1.0     1.0  
2  All right Joey, be nice. So does he have a hum...          2.0     1.0  
3                           Wait, does he eat chalk?          3.0     1.0  
4  Just, 'cause, I don't want her to go through w...          4.0     1.0  


In [56]:
dfc = df[df["author"] == "Chandler"]
dfc
for i in range(len(dfc)):
    print(dfc.index[i], dfc.iloc[i]["quote"])

2 All right Joey, be nice. So does he have a hump? A hump and a hairpiece?
6 Sounds like a date to me.
7 Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.
9 Then I look down, and I realize there's a phone... there.
11 That's right.
14 All of a sudden, the phone starts to ring. Now I don't know what to do, everybody starts looking at me.
16 Finally, I figure I'd better answer it, and it turns out it's my mother, which is very-very weird, because- she never calls me!
21 Cookie?
34 Sometimes I wish I was a lesbian... (They all stare at him.) Did I say that out loud?
41 And I just want a million dollars! (He extends his hand hopefully.)
53 (imitating the characters) Tuna or egg salad? Decide!
57 (re TV) Ooh, she should not be wearing those pants.
73 Please don't do that again, it's a horrible sound.
81 Ooh, this is a Dear Diary moment.
91 I'm sorry, I didn't catch your name. Paul, was it?
103 (deadpan) Yes, and we're very ex

4891 What?!
4893 (investigating) I turned it off. Mother of God, I turned it off!
4896 Nngghhh!!!!!!!
4946 Danielle! Hi! Uh- everybody, this is Danielle, Danielle, everybody.
4948 What are you doing here?
4950 ...I'm, I'm okay.
4952 That sounds good. I'll call you- or you call me, whatever...
4954 Okay.
4960 ...I dunno.
4962 Well, she seems very nice and everything, but that whole thing about her coming all the way down here, just to see if I was okay? I mean,... how needy is that?
4968 Wow, what a geek. They spent $69.95 on a Wonder Mop.
4975 Well, that's what we said about Joey...
4992 How about Joey... Pepponi?
4994 Joey... Switzerland?
4996 Joe...Joe...Joe...Stalin?
4998 Well, it does not ring a bell with me...
5000 Might wanna try Joseph.
5002 Oh yes! Bye Bye Birdie, starring Joseph Stalin. Joseph Stalin is the Fiddler on the Roof.
5047 She's not gonna make you wear one of those big plastic cones, is she?
5051 So what does this mean?
5054 I don't get it, I mean, you just got him. 

13637 Mar-haaaan.
13650 Well, that went well.
13652 Y'know, I was thinking about ah, asking her for her number.
13654 Whoa-ho, whoa! No, I was thinking about y'know for me, as a part of that whole getting over Janice thing you were talking about.
13656 Thanks. What, let me have her?! What do mean? Like if you didnt I wouldnt have a shot?
13658 Math!! Youre giving me math! All right, look y'know what, forget about it, you go for the girl, well see who gets her.
13667 Well now you have two. Hey, I am good at math.
13669 Hey! Well, Ive been preparing for that my entire life! Or something about you thats mean!
13679 Before!!
13698 Is everybody else seeing a troll doll nailed to a two by four?
13700 Okay, good.
13715 (to Margha) Hi.
13735 No-no-no-no, the games not over, were just switching teams.
13738 No ah, hold on a second Joe, where do Dutch people come from?
13740 And the other ah, Dutch people, they come on from somewhere near the Netherlands, right?
13745 Fine with me.
13751

18279 Uh-huh, first you tell us where you got the prettiest lace in all the land.
18283 (reading the ad) Stunning entertainment center. Fine, (pause) fine Italian craftsmanship. (Joey is very proud of himself)
18286 You want it?
18288 Ahh, Gepeto, $5,000 dollars? Are you insane?
18290 All right look, Im changing it to 50 bucks, or your best offer.
18292 (looks at him) Nooo.
18294 I cant believe you dont know what I do for a living!
18308 (interrupting) Let it go!!
18330 Yes, Joey has a very careful screening process. Apparently, not everyone is qualified to own wood and nails.
18348 Youll have to pardon my roommate, he wanted to marry this.
18350 Y'know, I, I really dont think we need a canoe.
18352 All right, just, just take the entertainment center, and then when you get home, throw the canoe away!
18367 Y'know what, I think this might be one of the times hes wrong.
18369 Oh-no, hes right.
18382 Im coming already!!
18414 OH MY GOD!!!
18416 Are you all right?!
18418 (lets him 

20773 Actually, its Miss Chinandolor Bong.
20783 Unfortunately that is correct.
20789 I majored in lightning rounds. All right, were gonna destroy you.
20791 Well, Im so confused as to what weve been doing so far
20794 Say 200?
20810 (interrupting) All right, hold on! If you win, we give up the birds.
20812 But if we win, we get your apartment.
20824 Okay. (Starts jumping around.)
20828 Dangerous Liaisons.
20832 Oh! (Whispers something in Joeys ear and then in Rosss ear.)
20835 Fancy.
20837 Fancy guest.
20848 Thanks man.
20865 You cant just call Monicas room.
20870 Oh, I would never bet this apartment. Its too nice.
20872 Hey!
20877 Yeah, but I bet it works.
20883 Well believe it baby!
20893 I knew it!
20895 Cool, girl roommate.
20924 Would you all stop yelling in our apartment! You are ruining moving day for us!
20937 What?! What-what is it?!
20939 I cant believe we live here!
20941 Okay, she is the star of the play. And she is my girlfriend! I get to have sex with the star 

...

43080 Did it turn into sand?
43082 (sliding up behind her) No. No, I wont. Do you know why I took all those lessons? See, for the first time I didnt want you to be embarrassed to be seen on the dance floor with some clumsy idiot.
43085 Why?
43132 (to the front desk clerk) Hi! Were checking out of the bridal suite.
43134 And Im the happiest guy in the world! (Monica goes and sits down in a huff.) Oh honey, come on dont be upset. We still have so much to look forward to!
43136 We got the honeymoon.
43138 The wedding pictures?
43140 Not the disposable cameras from the tables.
43142 Ill tell you what, I will go get them developed and you can go home.
43144 What? What did you take a picture of?
43146 Okay Ross has the cameras, has he checked out yet?
43148 Oh.
43151 Okay, well Im gonna get Ross, get the cameras, and get them developed. (Joey laughs again.) 32 Joe. Youre 32! (Exits)
43156 Hey. (He sees that Ross is packing all of the hotel toiletries) Soaps? Shampoos? Are you really 

48873 Oh no, its gonna be named after some snack or baked good isnt it?
48886 That was amazing.
48888 And if not, we got to do it on a bucket.
48971 He pulled a quarter out of my ear!
48976 Okay. Wow.
48995 Sure.
48997 That was you?! I thought it was Jack!
48999 What?! Why does she think that?
49001 Again, what?!
49003 As we all are at some point during the day.
49005 Ross had a ring?! And he was gonna propose?
49007 And you did it first?! This is gonna kill him! You know how much he loves to propose!
49009 Well, what did she say?
49011 Does Ross know?
49013 Well maybe you dont have to tell him anything.
49015 If you clear things up with Rachel then Ross never needs to find out, but you have to do it now before he hears about it and kicks your ass!
49018 Honey, weve been over this. I need to be facing the other way.
49020 Im so tired. (She starts kissing him.) Yeah okay, but no foreplay.
49055 Hello sir, you know Monica.
49106 Look, we cant stay in here forever.
49108 This is oka

52750 Oh good, 'cause I've already thought of 3... 4! I've just thought of a fourth (he goes towards the counter)
52758 I believe I read that somewhere!
52760 I respectfully disagree.
52762 Who cares? Nobody reads those things
52764 I don't have a page.
52774 Because I told everyone he slept with dinosaurs.
52776 Would you get that please? People have been calling to congratulate me all day.
52778 Hang up, hang up. And that was a great movie! (Monica hangs up) I'm so gonna get back at Ross... oh yeah, this will show him, here we go (starts typing something).
52780 Oh, you'll see my friend.
52782 (faking sympathy) And so young.
52784 Well, how you died was funny.
52786 It kills over one americans every year.
52788 You wanna talk about people's feelings? You should have heard how hurt professor Stern was yesterday when I told him I wouldn't be able to go with him to Key West!
52861 Hey, Ross, I just wanted to apologize... (­looks at laptop screen)..don't tell me you actually made those g

57359 Hi!
57362 We're just here to say goodbye, we're off to Ohio.
57366 And so incorrect!
57369 Uhm, thank you, but we're really trying not to get our hopes up.
57371 Yeah. I mean, this girl could decide against adoption or she could like another couple better..
57373 I can develop a condition in which I talk and talk and no one hears a word.
57376 So we're standing firm on the 'not getting our hopes up'?
57378 Yes, but...
57380 Oh, screw it, I'm gonna be a daddy!!
57417 Yeah. Just weird, you know. It's like: "Hi, I'm Chandler. May I have the human growing inside you?"
57419 You're gonna be great.
57424 Thank you so much for agreeing to see us.
57427 Ok.
57429 That's what our friends call us.
57433 (astonished) I don't think that's exactly...
57483 So, the fact that I am a doctor, and my wife’s a reverend, that’s important to you?
57486 Plus I thought the baby would be in good hands with a doctor!
57491 Yeah, wow.
57495 No-ot for me it doesn’t.
57499 Yeah, actually. So, you read a fil

59499 We're taking a break!
59501 Jumping on the bed?
59504 No, no, no, guys. She's right. We should get to work. I'll take stuff out of the closet, Joey you pack 'em and Ross you re-pack whatever Joey packs.
59506 (holding a pair of furry handcuffs) What the hell is this?
59509 I didn't know Monica had these!
59511 Why would she use them with Richard and not me? I can be kinky! I once did a naked dance for her... with scarves!
59522 Hey.
59535 I know, she's been such a big part of my life. And it feels like when Melrose Place got cancelled. (Ross and Joey looks puzzled) I mean... oh, forget it. I miss Melrose Place!
59539 Yes, Joe?
59557 Oh, yes. I decided to leave these out for you in case Richard stops by and you wanna engage on a little light bondage and moustache play!
59559 Oh yeah, right! Good luck getting another scarf dance from me!
59561 Really? Then what are they doing in our guestroom?
59563 Rachel... with handcuffs! Interesting! (he looks excited)
59565 (he doesn't look ex

### Number of uncaught scene changes (quote order skips by one)

In [54]:
prev_quote_order = -1
num_scene_changes = 0

for i in range(len(df)):
    curr_quote_order = df.iloc[i]["quote_order"]
    if curr_quote_order != prev_quote_order + 1:
        #print(i)
        num_scene_changes += 1
    prev_quote_order = curr_quote_order
num_scene_changes

839

### Extract pairs of previous dialogue and current dialogue

In [9]:
df_prev_dialogues = pd.DataFrame(columns=["prev_dialogues", "current_dialogue"])
num_words = 0
new_scene_start_idx = 0
prev_episode_id = 1

for i in tqdm(range(len(df))):
    #print(i)
    curr_episode_id = int(df.iloc[i]["episode_number"])

    # Don't use context from previous episode, start from this dialog
    if curr_episode_id != prev_episode_id:
        new_scene_start_idx = i

    # Don't use context from previous scene, start from this dialog
    if "Scene" in df.iloc[i]["quote"]:
        new_scene_start_idx = i + 1

    if df.iloc[i]["author"] == character_name:
        current_dialog_num = i #int(df.iloc[i]["quote_order"])
        current_dialog = df.iloc[i]["quote"]

        #prev_dialogues_range = [max(0, i - num_prev_dialogues) : current_dialog_num]
        prev_dialogues = " \n ".join(list((df.iloc[max(new_scene_start_idx, current_dialog_num - num_prev_dialogues) : current_dialog_num]["author"] + ": " + df.iloc[max(new_scene_start_idx, current_dialog_num - num_prev_dialogues) : current_dialog_num]["quote"]).values))
        prev_dialogues = re.sub("\(.*\)","", prev_dialogues)
        current_dialog = re.sub("\(.*\)","", current_dialog)

        df_prev_dialogues.loc[i] = [prev_dialogues, current_dialog]
        num_words += len(prev_dialogues.split(" "))

    prev_episode_id = curr_episode_id

df_prev_dialogues.reset_index(inplace=True)
df_prev_dialogues.drop('index', axis=1, inplace=True)
df_prev_dialogues.head(2)

100%|██████████| 60291/60291 [01:12<00:00, 829.50it/s] 


,prev_dialogues,current_dialogue
0,Monica: There's nothing to tell! He's just som...,"All right Joey, be nice. So does he have a hum..."
1,Monica: There's nothing to tell! He's just som...,Sounds like a date to me.


In [27]:
for i in range(len(df_prev_dialogues)):
    print(i, df_prev_dialogues.iloc[i]["current_dialogue"])

0 All right Joey, be nice. So does he have a hump? A hump and a hairpiece?
1 Sounds like a date to me.
2 Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.
3 Then I look down, and I realize there's a phone... there.
4 That's right.
5 All of a sudden, the phone starts to ring. Now I don't know what to do, everybody starts looking at me.
6 Finally, I figure I'd better answer it, and it turns out it's my mother, which is very-very weird, because- she never calls me!
7 Cookie?
8 Sometimes I wish I was a lesbian...  Did I say that out loud?
9 And I just want a million dollars! 
10  Tuna or egg salad? Decide!
11  Ooh, she should not be wearing those pants.
12 Please don't do that again, it's a horrible sound.
13 Ooh, this is a Dear Diary moment.
14 I'm sorry, I didn't catch your name. Paul, was it?
15  Yes, and we're very excited about it.
16 It's a beautiful thing.
17 I would have to say that is an 'L'-shaped bracket.
18 I hav

689 And this has nothing to do with the fact that he needs a note to get out of gym.
690 Tyrannosaurus!
691 Work people? Nobody told me.
692 You know, I don't get this. A month ago, these people were my friends. You know, just because I'm in charge doesn't mean I'm a different person.
693 You think I should?
694 Okay.
695 Okay.
696 I Think last night was great. You know, the Karaoke thing. Tracy and I doing Ebony and Ivory.
697 What?
698 Then, I don't get it.
699 They do?
700 I just wan't to...
701 But I just wa...
702 Thanks. Listen, about the weekly numbers, I'm gonna need them on my desk by nine o'clock.
703 You have to give 'em something, you know. Okay, now that was Gerston, Santos, and who's the guy with the moustache?
704 Petrie, right, right. Okay, some people gonna be working this weekend.
705  Hey Ross, look what I've got going here.
706 Yeah, like in a cab...
707 Okay, hating this.
708 Here we go, here we go.
709 Yes, Joey's made arrangements to have his baby in a movie from

1234  Hey, so can you make it on Friday?
1235 Well apparently Albert has no friends. He's very excited about the bachelor party though. I think actually the only reason he's getting married is so he can see a stripper.
1236 Well, if the magician can open my beer with his but cheeks, then all right.
1237 Don't do anything. Keep it inside. Learn how to hide your feelings!  Don't cry outloud.
1238 I want to say you but, that seems like such an easy answer.
1239 I'll get you the Cliff Notes.
1240 The abridgment.
1241 ....three years of modern dance with Twila Tharp! Five years with the American Ballet Theater?!
1242 Oh no, no, no, no.
1243 All right, I have to get that, but no-no.  So would I, would I have to provide the grapes?
1244 Okay, we have our stripper. A miss Crystal Chandelier.
1245 To yourself.  Hey-hey-hey, y'know what that's pretty good.
1246 Bye-bye.
1247 What, what, what?!
1248 You really think that is what he meant?
1249 Look, don't you see what's happening here. Instead of

1920 Oh no-no-no. This is amazing. 
1921 Well, yknow Im 29. I mean who needs a savings account.
1922 Oh, I dont know.
1923 You mean with Casey.
1924 Or Casey.
1925 Could be Casey.
1926 Consider Casey.
1927 That all the pieces of my life are falling right into place!
1928 I kissed Kathy.
1929 No. Is there anyway, anyway you think hell understand this?
1930 Oh, dont say that! Dont say that. Thats not true. Is it?
1931 Then why didnt you tell me to do that?!!
1932 God!!  What am I gonna do?!
1933 Why?! Why do I have to tell him?!
1934 Yeah, I know.
1935 Hey!
1936 Is that a real thing?
1937 Listen ah, Joe, I-I need to, I need to talk to you about something.
1938 Its-its about Kathy. Umm, uh, I like her. I like her a lot actually.
1939 Yeah.
1940 What?
1941 Oh uh, when?
1942 Oh man, I am so sorry. Are, are you okay?
1943 Yes, but I-I uh, dont have too.
1944 Yeah?
1945 Well, I thought that would be the best thing to do.
1946 See uh, thats-thats actually what I wanted to talk to 

2495  All right! Lets do it!
2496 Okay, cause thats not gonna get annoying.
2497 Hey, yknow what? I was wrong.
2498 Hey! 
2499 Oh. 
2500  Here you go Phoebe! Here you go Phobo! Phewbedo! Phaybobo.
2501 I am, Im ignoring you.
2502 Oh, what the hell is that?
2503 Okay, if you see a little version of me in there? Kill it!
2504 Okay. Listen-listen, were not gonna have to walk this way the entire time are we?
2505 I think its great. Its great. Yknow, theyre thinking of changing the name of this place.
2506 To Put the Camera Away!!!
2507 Well, I dont have to buy that, "Im with stupid" T-shirt anymore.
2508 All right, look, youre not really gonna buy that are you? Dont you think youve embarrassed me enough for one day?
2509 How can I answer that when Im pretending I dont know you?
2510 Oh really? Then how come no one here is wearing them?
2511 All right, look, if you insist on wearing that, in public, yknow, youre gonna spend the rest of the afternoon all by yourself.
2512 

...

4437 Hi.
4438 Richard! No one supposed to know about us!  See I, did it again.
4439 Yeah, Ill sit down. 
4440 Yes. 
4441 I got a good one, I got a good one! I once walked in on both my parents making love to the same guy.
4442 What?!
4443  Before you say anything, have we got a story for you! Guess who we bumped into at dinner!
4444 Richard!
4445 Oh you knew that. Good!
4446 For camp!
4447 Hey, you have to forget about Elizabeth. I mean if youre not careful you may not get married at all this year!
4448  What did you guys just do?!
4449 Richard was there so I couldnt do it!
4450 Im gonna do it tomorrow yknow, and-and surprise her, but now youve ruined it!
4451 Who walks into a room and asks to see a persons hands?!
4452 This is terrible. What am I going to do?
4453 Thats right, I can throw her off. I can make her think marriage is the last thing on my mind.
4454 I can do that, Ive had 30 years of practice.
4455 Okay, okay, here she comes!  How do I look? Do I look like a guy w

5071 No! No! No!
5072 Theyre in my bag over there. 
5073 Uh, could you leave me one?
5074 Yeah.
5075 No I-I-Im fine.
5076 What?
5077 Who did you originally want to hook up with?
5078  So you came to the room looking for Joey? Did you ever in-intend on telling me about this?
5079 Oh, its not important? Its not important?! If it wasnt for a brides maid youd be marrying him  not me!
5080 I dont believe it. The most romantic night of my life and Im runner up.
5081  Look there is no way youre doing this wedding now. Okay?
5082 Okay, its just weird! Okay? I dont want to be standing their saying my vows and then having the mental image of you and Monica! I-I-I needI dont know what I need. I need a walk.
5083 It is to me. You wanted to sleep with Batman, and instead you had to settle for Robin. 
5084 Hey. Do you want this scone?  It came for me but it would probably rather sleep with you!
5085 Look Joe, I know you wanted to do the wedding
5086 How can I not be upset? Okay? I fina

5687 Do you?
5688 In London?
5689 Yes. When Monica and I were in London, we were both in London.
5690 If that.Scene: Central Perk, Phoebe and Rachel are sitting on the couch. Ross sits down.]
5691 Hey.
5692 Sex on the balcony?
5693 What is it?
5694 Honey, I dont like baths! Could you draw me a picture of us having sex on the balcony?
5695 Well, this does butch it up a bit.
5696 All right, if I do this, can we at least discuss sex on the balcony?
5697  All right, this isnt so bad. I like the flower smell! Which is okay, because Ive got my boat.
5698 Oh my God.
5699 Im going to need a bigger boat.Scene: The Doctors Office, the doctor is writing something as Rachel is on the table, and Ross is standing.]
5700 Im in the bathroom, can you come in here? I think theres something wrong.
5701 Im in the bathtub.
5702 I drew my own bath, but I did it wrong! The waters tepid. The salt didnt dissolve and is now lodged places. And the scents I used dont compliment each other. Eucalyptus 

6436 Oh, no, no, I'm an intern, just like you guys except for the tie, the briefcase and the fact that I can rent a car.
6437 Yeah, well, I'm kinda heading into a new career direction and, you know, you gotta start at the bottom.
6438 Right. Look, I know I'm a little bit older than you guys, but it's not like I'm Bob Hope 
6439 The comedian? USO?!
6440  Honey, I'm old!
6441 I am so much older than these other interns. I can't compete with them.
6442 Yes, but I don't think life experience with these. 
6443 They're these prototype sneakers and come up with ideas on how to sell them which I can't do because no self-respecting adult would ever where these.
6444 What am I supposed to do with these?
6445 Well, first of all, they're not called "sneakers" anymore. Apparently, they're called "kicks" or "skids" and I think I heard somebody say "slorps".  Would you help me try to sell these?
6446 No, I don't think that's the kinda thing they're looking for.
6447 Hey!
6448 Putting on the sneaker

7186  Sure, nice.
7187 You're right, by saying "nice" I'm virtually licking her.
7188 Are you asking permission to break the pact?
7189 Awesome, the name really stands out.
7190 Man, we're gonna rock that Asian student union!
7191 Hey!
7192 You know, our band is playing on Friday.
7193 Fresh!
7194 Mint!
7195 I know it. You know, I'm totally gonna ask her out.
7196 I said it first, bro.
7197  Look, if you did...
7198 Totally. I had sex in High school...
7199 All right, I'd say we make a pact. Neither of us will go out with Missy Goldberg.
7200 All right, so that's Missy Goldberg, Phoebe Cates and Molly Ringwald, who neither of us can go out with.
7201 Oh, and Sheena Easton. But we probably couldn't get her anyway.
7202 Well, I officially give you permission to break the pact.
7203 Your hands are shaking.
7204 . That didn't make us sound gay at all!
7205 Ross, that was 16 years ago!
7206 I believe the foundation of our friendship was unfortunate hair.  All right, look, if we're really go

In [11]:
print(f"Number of words in context dialogues: {num_words}")
print(f"Number of Character dialogues: {len(df_prev_dialogues)}")

num_words_content_message = len(content_system.split(" ")) * len(df_prev_dialogues)
print(f"Number of words to batch input: {num_words + num_words_content_message}")

Number of words in context dialogues: 1032393
Number of Character dialogues: 7488
Number of words to batch input: 1998345


### Write data in OpenAI Batch API file format


In [12]:
# OpenAI Batch API file format
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}

{'custom_id': 'request-1',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-3.5-turbo-0125',
  'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'},
   {'role': 'user', 'content': 'Hello world!'}],
  'max_tokens': 1000}}

In [13]:
def get_json_schema_for_response_format():
    schema = {
        "type": "object",
        "properties": {
            "context": {"type": "string"},
            "dialog": {"type": "string"},
        },
        "required": ["context", "dialog"],
        "additionalProperties": False
    }

    json_schema = {
        "name": "ContextDialogue",
        "schema": schema,
        "strict": True
    }

    response_format = {
        "type": "json_schema",
        "json_schema": json_schema
    }

    return response_format

In [14]:
custom_id = 0
df_batch_input = pd.DataFrame(columns=["custom_id", "method", "url", "body"])

for i in tqdm(range(len(df_prev_dialogues))):
    custom_id_str = "request-" + str(custom_id)
    body = {}
    body["model"] = "gpt-4o-mini"

    messages = []
    message_system = {}
    message_system["role"] = "system"
    message_system["content"] = content_system

    message_user = {}
    message_user["role"] = "user"
    message_user["content"] = df_prev_dialogues.iloc[i]["prev_dialogues"]

    messages.append(message_system)
    messages.append(message_user)
    body["messages"] = messages

    body["response_format"] = get_json_schema_for_response_format()
    body["max_tokens"] = 500
    
    df_batch_input.loc[i] = [custom_id_str, "POST", "/v1/chat/completions", body]
    custom_id += 1

100%|██████████| 7488/7488 [00:20<00:00, 363.91it/s]


In [15]:
import json
# Convert DataFrame to JSONL (this will escape forward slashes)
#df_batch_input.to_json('output.jsonl', orient='records', lines=True)

# Convert to JSON string without escaping forward slashes
#json_str = json.dumps(df_batch_input.to_dict(orient='records'), ensure_ascii=False).encode('utf8')
#batch_data = json.loads(json_str)
batch_data = df_batch_input.to_dict(orient='records')

In [16]:
midpoint = len(batch_data)//2
batch_data_part1 = batch_data[:midpoint]
batch_data_part2 = batch_data[midpoint:]
len(batch_data_part2)

3744

In [17]:
# Write to a JSONL file
with open(batch_ip_path_part1, "w", encoding="utf-8") as f:
    for i, line in enumerate(batch_data_part1):
        json.dump(line, f, ensure_ascii=False)  # Serialize dict as JSON
        if i != len(batch_data) - 1: f.write("\n")  # Newline for JSONL format

In [18]:
# Write to a JSONL file
with open(batch_ip_path_part2, "w", encoding="utf-8") as f:
    for i, line in enumerate(batch_data_part2):
        json.dump(line, f, ensure_ascii=False)  # Serialize dict as JSON
        if i != len(batch_data) - 1: f.write("\n")  # Newline for JSONL format

In [19]:
print(json.dumps(line, indent=2))

{
  "custom_id": "request-7487",
  "method": "POST",
  "url": "/v1/chat/completions",
  "body": {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "system",
        "content": "You are given a few dialogues from the famous TV sitcom FRIENDS. You have two tasks. First, you have to summarize the dialogues into context without any names of characters. Second, you have to then write a dialogue as an appropriate response to the given context, the dialogue should be in the style of Chandler Bing, one of the characters from FRIENDS. Keep in mind that Chandler Bing\u2019s humor is marked by a unique blend of sarcasm, self-deprecation, and quick wit. He tends to make jokes that deflect serious or emotional moments, often using his dry, sarcastic tone.  His style is heavily reliant on irony, often delivering punchlines that are deliberately over-the-top or nonsensical. Extract these two things and return them as structured output specified as Pydantic schema."
      },
     

In [96]:
#!pip install openai
#from openai.lib._parsing._completions import type_to_response_format_param
#from openai.lib._pydantic import to_strict_json_schema
#type_to_response_format_param(ContextDialogue)

### Upload file to OpenAI batch API

In [16]:
import getpass
import os

In [3]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


In [ ]:
from openai import OpenAI
client = OpenAI()

# Upload file
batch_input_file = client.files.create(
  file=open("../datasets/friends/batch_file_friends.jsonl", "rb"),
  purpose="batch"
)

In [ ]:
# Create Batch
batch_input_file_id = batch_input_file.id
client.batches.create(
  input_file_id=batch_input_file_id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [ ]:
# Get status
const batch = client.batches.retrieve(batch_input_file_id)
print(batch)

In [19]:
dfi = pd.read_json(batch_ip_path_part1, lines=True)
dfi.head(2)

,body,custom_id,method,url
0,"{'model': 'gpt-4o-mini', 'messages': [{'role':...",friends-0,POST,/v1/chat/completions
1,"{'model': 'gpt-4o-mini', 'messages': [{'role':...",friends-1,POST,/v1/chat/completions


In [8]:
dfo = pd.read_json(batch_op_path_part1, lines=True)
dfo.head(2)

,custom_id,error,id,response
0,friends-0,NaN,batch_req_67b449435480819089e6a849cbe87e60,"{'status_code': 200, 'request_id': '9ac291a7b9..."
1,friends-1,NaN,batch_req_67b4494377448190a626bb16325f8515,"{'status_code': 200, 'request_id': 'ee24801afc..."


### Get chat completions from OpenAI API

In [ ]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class ContextDialogue(BaseModel):
    context: str
    dialog: str

In [ ]:
#response = client.chat.completions.create(
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "developer",
            "content": """You are given a few dialogues from the famous TV sitcom FRIENDS. You have two tasks.
                          First, you have to summarize the dialogues into context without any names of characters.
                          Second, you have to then write a dialogue as an appropriate response to the given context,
                          the dialogue should be in the style of Chandler Bing, one of the characters from FRIENDS.
                          Keep in mind that Chandler Bing’s humor is marked by a unique blend of sarcasm, self-deprecation, and quick wit.
                          He tends to make jokes that deflect serious or emotional moments, often using his dry, sarcastic tone.
                          His style is heavily reliant on irony, often delivering punchlines that are deliberately over-the-top or nonsensical.
                          Extract these two things and return them in JSON format."""
        },
        {
            "role": "user",
            "content": """Phoebe: Ooh! Oh!  \n Ross: No, no don\'t! Stop cleansing my aura! No, just leave my aura alone, okay? \n Phoebe: Fine! Be murky! \n Ross: I\'ll be fine, alright? Really, everyone. I hope she\'ll be very happy. \n Monica: No you don\'t. \n Ross: No I don\'t, to hell with her, she left me! \n Joey: And you never knew she was a lesbian... \n Ross: No!! Okay?! Why does everyone keep fixating on that? She didn\'t know, how should I know? \n Chandler: Sometimes I wish I was a lesbian...  Did I say that out loud? \n Ross: I told mom and dad last night, they seemed to take it pretty well. \n Monica: Oh really, so that hysterical phone call I got from a woman at sobbing 3:00 A.M., "I\'ll never have grandchildren, I\'ll never have grandchildren." was what? A wrong number? \n Ross: Sorry. \n Joey: Alright Ross, look. You\'re feeling a lot of pain right now. You\'re angry. You\'re hurting. Can I tell you what the answer is? \n Joey: Strip joint! C\'mon, you\'re single! Have some hormones! \n Ross:"""
        }
    ],
    response_format=ContextDialogue,
)

print(response.choices[0].message.content);

{"context":"A character is upset about a breakup, feeling angry and hurt, while others offer mixed responses ranging from sarcastic quips to genuine concern. Some attempt to comfort him while others make jokes about his situation, leading to a humorous yet awkward moment.","dialog":"Could we have a little less aura and a little more chips and guac? I mean, if we're cleansing anything, let's cleanse the tacos, am I right?"}


In [ ]:
import json
json.loads(response.choices[0].message.content)

{'context': "In this scene, a group of friends is dealing with the aftermath of one friend's recent breakup. One friend is attempting to cleanse another friend's aura, but he insists on being left alone. Throughout the conversation, they discuss feelings of anger and sadness related to the breakup, while a couple of the friends poke fun by offering sarcastic or outrageous suggestions on how to cope with the situation, like going to a strip joint. Additionally, there's humor derived from one friend's unexpected and humorous desire to be part of the LGBTQ+ community.",
 'dialog': "Could I BE any more heartbroken? I mean, why go to a strip joint when I can just gather my friends and wallow in self-pity while eating an entire pizza by myself? That's practically a family tradition!"}